In [1]:
import soundfile as sf
import sounddevice as sd
import numpy as np
import sys
import multiprocessing
import time
from scipy import signal

from queue import Queue
from onnx import ServiceModel


In [2]:

sr = 16000
ch = 1
subtype = 'PCM_16' # sample_width 2byte(16bit)
sw = 2 # sample_width

model = ServiceModel()


In [3]:

def service(sig):
    print(model.one_shot(np.array(sig).flatten()))


In [4]:
global running, threshold
running = True
sig_flag = False

q = Queue()

def rms(array):
    return np.sqrt( np.mean( np.array(array) **2 ) )

def mean_abs(array):
    return np.mean( np.abs(array) )

def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    q.put(indata.copy())
    
### new
    
ft = signal.firwin(1500,[300,3000],pass_zero=False,fs=sr)

def freq_filter(array):
    if len(array) > 1600:
        return signal.lfilter(ft,[1.0],array)
    else:
        return None
   
###

def get_q():
    array = []
    while not q.empty():
        array.extend(q.get())
    if len(array) > 1:
        return np.concatenate(array)
    else:
        return np.array(array)
    

In [6]:
sigma_rule_num = 4
pause_cnt = 10
sig_flag = False
speech_sig = np.zeros(1,dtype=np.float32)
try:
    with sd.InputStream(samplerate=sr, channels=ch, callback=callback):
        time.sleep(2)
    sig = get_q()
    print(sig.shape, sig.dtype)
    start_time = time.time()
    sig = freq_filter(sig) ### change
    print(time.time()-start_time)
    
    ## 시그마 규칙
    sig = np.abs(sig)
    threshold = np.mean(sig) + (np.std(sig) * sigma_rule_num )
    
    print("init complete")
    print("threshold :",threshold)
    with sd.InputStream(samplerate=sr, channels=ch, callback=callback):
        sig = np.zeros(1,dtype=np.float32)
        tmp = np.zeros(1,dtype=np.float32)
        print(sig.shape, sig.dtype)
        print("running start")
        
        while running:
            # time.sleep(1)
            sig = np.concatenate((sig, get_q()))
            # start_time = time.time()
            harmonic_sig = freq_filter(sig) ### change
            # print(time.time()-start_time)
            if harmonic_sig is None:
                continue
            
            if rms(harmonic_sig) > threshold:
                print("saying")
                start_time = time.time()
                # time.sleep(1)
                speech_sig = np.concatenate((speech_sig, tmp))
                sig_flag = True
                cnt = 0
            else:
                if sig_flag:
                    if cnt < pause_cnt:
                        speech_sig = np.concatenate((speech_sig,tmp))
                        cnt += 1
                    else:
                        print("STT")
                        print(time.time()-start_time)
                        service(speech_sig.astype(np.float32))
                        speech_sig = np.zeros(1,dtype=np.float32)
                        sig_flag = False
            tmp = sig
            sig = np.zeros(1,dtype=np.float32)
            
except KeyboardInterrupt as ke:
    print("Recording finished")

(31616,) float32
0.011035919189453125
init complete
threshold : 0.001384356370118224
(1,) float32
running start
saying
saying
saying
saying
STT
1.1409995555877686
파
saying
saying
saying
saying
saying
saying
saying
saying
saying
STT
1.1399986743927002
파
saying
saying
saying
saying
STT
1.1420011520385742
파
saying
saying
saying
saying
saying
saying
saying
saying
STT
1.1470348834991455
파
Recording finished


In [ ]:
q.get()

In [ ]:

global running
running= True

try:
    with mic:
        procs = []
        print('start running')
        while running:
            start_time = time.time()
            # rec.adjust_for_ambient_noise(mic)
            sig = rec.listen(mic).get_wav_data(sr,2)
            sig_time = time.time()
            print(sig_time-start_time)
            
            data_s16 = np.frombuffer(sig, dtype=np.int16, count=len(sig)//2, offset=0)
            float_data = (data_s16 * 0.5**15).astype(np.float32)
            # sd.play(float_data,sr)
            # print(type(float_data))
            # print(float_data.dtype)
            data_cvt_time = time.time()
            print(data_cvt_time-sig_time)
            
            service(float_data)
            # p = multiprocessing.Process(target=service,args=(sig,))
            # p.start()
            # procs.append(p)
            
            print(time.time()-data_cvt_time)

            # time.sleep(5)

except KeyboardInterrupt as ke:
    print("finished")
except Exception as e:
    print(e)
    
for p in procs:
    p.join()
    
print("Program end")

start running
9.67878007888794
0.000997781753540039
보 있늤는 방 붕지 시켜 줘
3.848050117492676
11.777889251708984
0.001058816909790039
보 있느는벙정지 시켜ㅗ 어떵 어 ㅗ까
1.6419508457183838
3.3970539569854736
0.0010578632354736328
지ㅔ가 보입언중 대처로 으인지 궁금해
1.6899504661560059
9.917074918746948
0.0
어어
0.8709938526153564
10.367519617080688
0.001911163330078125
보고 있느넝 사덩지 시쾨도 더 죾아지날시가 아때 나 대 시넌드미 조이버 주더첼오뢰는지 궁그면 지금나잔 취침등 꺼 줘
4.793968439102173
51.718247175216675
0.0019822120666503906
고있느네 가등ㄴ 싴교판 주가에너씨가데나 ㄷ 신 정뇐 ㅜ 입어 주 되즈롤는ㅈ ㅇ그미 ㅣㅇㄴ가나시ㅣ우꺼 주 입언 싶엉ㄴ감주ㅔ름지ㅔ겅 저 입어 종아날씨종ㅎ알ㅂ아 봦 지 ㅇㄴㅁ씨는좋아아잔지 알아 봐 조 간난거에 ㄴ 버데 날씹ㅇ기 이거ㅏㄸㄴ기리 까자 아꺼워 강에부일 좀 기고 싶담유일블보렬 줄래도ㅓ스트 ㅇ려 ㅈ래 즐래 즐겨찾기 ㅇ목으로 이동
27.606492280960083
54.902129888534546
0.1030573844909668
장남부오늘 날씨 브 리핀 거떤즈리 가자 가까워판랠 불 점기고 싶다 금면밸 더 퐁 줄래ㅗ스트려럴 조ㅜㅜㄹ 수 있드겨잡가목으로 있도 ㄴ자가까우 도로 원활하는 곳 알ㄹ디오북 취치모드 있진재 도로 교통 사어떤지 알ㄹ 주가 교토 사학안 어떻게된 다구웠으니 가도없어 ㅇ풀금 주가강 어제에 비레 몇 퍼올라는질ㄹ어어 전정리 켜 줘
65.82548213005066
finished
Program end
